In [6]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)
from src.main import*
import time as tm
d = pd.read_csv('../data/reedfrogs.csv', sep = ';')
d["tank"] = np.arange(d.shape[0])
formula = dict(main = 'y ~ Binomial(total_count = density, logits = p)',
               likelihood = 'p ~ alpha[tank]', 
               prior = 'alpha ~ Normal(a_bar, sigma)',
               prior1 = 'a_bar ~ Normal(0.,1.5)',
               prior2 = 'sigma ~ Exponential(1)'
               )

start = tm.time()   
m13_2 = model(formula, d, float=32)
m13_2.fit(observed_data = dict(y =d.surv.astype('float32').values), num_chains= 1, num_results=500)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-17 14:59:19.227530: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-17 14:59:19.227578: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-17 14:59:19.227591: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-17 14:59:19.227754: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-17 14:59:19.227766: I tensorflow/core/common_runtime/gpu/gpu

BI took: 2.3843 seconds


In [5]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)
from src.main import*
import time as tm

import os
import warnings

import arviz as az
import matplotlib.pyplot as plt
import pandas as pd

import jax.numpy as jnp
from jax import lax, random
from jax.scipy.special import expit

import numpyro
import numpyro.distributions as dist
from numpyro.diagnostics import effective_sample_size
from numpyro.infer import MCMC, NUTS, Predictive

az.style.use("arviz-darkgrid")
numpyro.set_platform("cpu")
numpyro.set_host_device_count(4)

reedfrogs = pd.read_csv("../data/reedfrogs.csv", sep=";")
d = reedfrogs
d["tank"] = jnp.arange(d.shape[0])

dat = dict(S=d.surv.values, N=d.density.values, tank=d.tank.values)

def model(tank, N, S):
    a_bar = numpyro.sample("a_bar", dist.Normal(0, 1.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    a = numpyro.sample("a", dist.Normal(a_bar, sigma), sample_shape=tank.shape)
    logit_p = a[tank]
    numpyro.sample("S", dist.Binomial(N, logits=logit_p), obs=S)

start = tm.time()  
m13_2 = MCMC(NUTS(model), num_warmup=500, num_samples=500, num_chains=1)
m13_2.run(random.PRNGKey(0), **dat)
end = tm.time()    
print(f"NumpyPro took: {end - start:.4f} seconds")

/home/sosa/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
sample: 100%|██████████| 1000/1000 [00:00<00:00, 1154.88it/s, 7 steps of size 4.49e-01. acc. prob=0.88]


NumpyPro took: 2.2508 seconds


In [7]:
import jax
import jax.numpy as jnp
from jax import random, jit
from tensorflow_probability.substrates import jax as tfp
from tensorflow_probability.substrates.jax.distributions import JointDistributionNamedAutoBatched as JDNAB
tfd = tfp.distributions
d = pd.read_csv('../data/reedfrogs.csv', sep = ';')
d["tank"] = np.arange(d.shape[0])
# Define the model using JointDistributionNamed
m = {}
m['alpha'] = lambda a_bar, sigma: tfd.Sample(tfd.Normal(a_bar, sigma), sample_shape=48)
m['a_bar'] = tfd.Normal(0., 1.5, name='a_bar')
m['sigma'] = tfd.Exponential(1., name='sigma')
m['y'] = lambda alpha: tfd.Independent(
    tfd.Binomial(
        total_count=jnp.array(d.density.values, dtype=jnp.float32),
        logits=jnp.squeeze(jnp.take(alpha, jnp.array(jnp.array(d.tank.values, dtype=jnp.float32), dtype=jnp.int32), axis=-1)),
        name='main'), reinterpreted_batch_ndims=1)

dist = JDNAB(m)

observed_data = d.surv.astype('float32').values
observed_data_jax = jnp.array(observed_data)  # Convert observed_data to JAX array

n_chain = jax.local_device_count()
init_key, sample_key = random.split(random.PRNGKey(0))
init_params = dist.sample(seed=jnp.array(init_key, dtype=jnp.uint32))
init_params.pop('y')
init_params = list(init_params.values())
init_params

def target_log_prob(*params):
    param_dict = {}
    keys = dist._flat_resolve_names()
    for i, key in enumerate(keys):
        if key != 'y':
            param_dict[key] = params[i]
    param_dict['y'] = observed_data_jax
    return dist.log_prob(param_dict)

@jit
def run_chain(key, state):
    kernel = tfp.mcmc.NoUTurnSampler(target_log_prob, 1e-3)
    return tfp.mcmc.sample_chain(500,
                                  current_state=state,
                                  parallel_iterations = 4,
                                  kernel=kernel,
                                  trace_fn=lambda _, results: results.target_log_prob,
                                  num_burnin_steps=500,
                                  seed=key)

start = tm.time() 
states, log_probs = run_chain(sample_key, init_params)
end = tm.time()    
print(f"TF took: {end - start:.4f} seconds")
states

TF took: 1.8506 seconds


[Array([1.5598401, 1.4051214, 1.5435513, 1.3364301, 1.4077877, 1.6272001,
        1.3198097, 1.3351039, 1.7368073, 1.342189 , 1.5906305, 1.4484773,
        1.4479122, 1.5700964, 1.5952141, 1.2353877, 1.9160602, 1.8255293,
        1.7152565, 1.4698378, 1.545837 , 1.6804879, 1.2200567, 1.6066158,
        1.5954791, 1.4895014, 1.4060465, 1.6175648, 1.9758253, 1.8574038,
        1.3843775, 1.4053863, 1.8166069, 1.7893407, 1.6175308, 1.6497587,
        1.6781268, 1.6529273, 1.7366393, 2.0168731, 1.5380782, 1.7787638,
        1.6211838, 1.4607064, 1.5490615, 1.2850691, 1.4371417, 1.5508647,
        1.5695531, 1.2991209, 1.4920267, 1.29972  , 1.3155978, 1.365459 ,
        1.5790433, 1.3226976, 1.7409965, 1.3669976, 1.472895 , 1.4853331,
        1.3712497, 1.4260033, 1.5566075, 1.6618371, 1.5025378, 1.4881272,
        1.2364163, 1.7605934, 1.5452589, 1.6366322, 1.6049116, 1.7047176,
        1.5908494, 1.8317045, 1.3525642, 1.7843983, 1.3290792, 1.2058443,
        1.2664813, 1.5034506, 1.843442

In [15]:
sample_key

Array([2718843009, 1272950319], dtype=uint32)

In [16]:
from jax import device_get, jit, lax, local_device_count, pmap, random, vmap
rng_keys  = random.split(random.PRNGKey(0), 4)
traces = pmap(run_chain(sample_key, init_params))(rng_keys)

TypeError: Expected a callable value, got StatesAndTrace(
  all_states=[Array([1.5598401, 1.4051214, 1.5435513, 1.3364301, 1.4077877, 1.6272001,
           1.3198097, 1.3351039, 1.7368073, 1.342189 , 1.5906305, 1.4484773,
           1.4479122, 1.5700964, 1.5952141, 1.2353877, 1.9160602, 1.8255293,
           1.7152565, 1.4698378, 1.545837 , 1.6804879, 1.2200567, 1.6066158,
           1.5954791, 1.4895014, 1.4060465, 1.6175648, 1.9758253, 1.8574038,
           1.3843775, 1.4053863, 1.8166069, 1.7893407, 1.6175308, 1.6497587,
           1.6781268, 1.6529273, 1.7366393, 2.0168731, 1.5380782, 1.7787638,
           1.6211838, 1.4607064, 1.5490615, 1.2850691, 1.4371417, 1.5508647,
           1.5695531, 1.2991209, 1.4920267, 1.29972  , 1.3155978, 1.365459 ,
           1.5790433, 1.3226976, 1.7409965, 1.3669976, 1.472895 , 1.4853331,
           1.3712497, 1.4260033, 1.5566075, 1.6618371, 1.5025378, 1.4881272,
           1.2364163, 1.7605934, 1.5452589, 1.6366322, 1.6049116, 1.7047176,
        

In [6]:
observed_data = d.surv.astype('float32').values
observed_data

array([ 9., 10.,  7., 10.,  9.,  9., 10.,  9.,  4.,  9.,  7.,  6.,  7.,
        5.,  9.,  9., 24., 23., 22., 25., 23., 23., 23., 21.,  6., 13.,
        4.,  9., 13., 20.,  8., 10., 34., 33., 33., 31., 31., 35., 33.,
       32.,  4., 12., 13., 14., 22., 12., 31., 17.], dtype=float32)

In [17]:
import jax.numpy as jnp
from jax import grad
from jax import jit
from jax import random
from jax import value_and_grad
from jax import vmap
from tensorflow_probability.substrates import jax as tfp
from sklearn import datasets
tfd = tfp.distributions
tfb = tfp.bijectors
tfpk = tfp.math.psd_kernels

def mymodel():
    a_bar = yield tfd.Sample(tfd.Normal(0., 1.5), sample_shape=1)
    sigma = yield tfd.Sample(tfd.Exponential(1.), sample_shape=1)
    alpha = yield tfd.Sample(tfd.Normal(a_bar, sigma), sample_shape=48)
    # Convert data to JAX arrays
    density_jax = jnp.array(d.density.values, dtype=jnp.float32)
    tank_jax = jnp.array(d.tank.values, dtype=jnp.float32)
    yield tfd.Independent(tfd.Binomial(total_count=density_jax,
                                        logits=jnp.squeeze(jnp.take(alpha, jnp.asarray(tank_jax, dtype=jnp.int32), axis=-1)),
                                        name='main'),
                          reinterpreted_batch_ndims=1)




dist = tfd.JointDistributionCoroutine(mymodel)

observed_data = d.surv.astype('float32').values
observed_data

def target_log_prob(*params):
  return dist.log_prob(params + (observed_data,))

init_key, sample_key = random.split(random.PRNGKey(0))
init_params = tuple(dist.sample(seed=jnp.array(init_key, dtype=jnp.uint32))[:-1])

@jit
def run_chain(key, state):
  kernel = tfp.mcmc.NoUTurnSampler(target_log_prob, 1e-3)
  return tfp.mcmc.sample_chain(500,
      current_state=state,
      kernel=kernel,
      trace_fn=lambda _, results: results.target_log_prob,
      num_burnin_steps=500,
      seed=key)
start = tm.time() 
states, log_probs = run_chain(sample_key, init_params)
end = tm.time()    
print(f"TF took: {end - start:.4f} seconds")

TF took: 2.0032 seconds


In [20]:
# Parallel jax!!!!!!!!!!!!!!!!
import jax.numpy as jnp
from jax import grad
from jax import jit
from jax import random
from jax import value_and_grad
from jax import vmap
from tensorflow_probability.substrates import jax as tfp
from sklearn import datasets
tfd = tfp.distributions
tfb = tfp.bijectors
tfpk = tfp.math.psd_kernels

def mymodel():
    a_bar = yield tfd.Sample(tfd.Normal(0., 1.5), sample_shape=1)
    sigma = yield tfd.Sample(tfd.Exponential(1.), sample_shape=1)
    alpha = yield tfd.Sample(tfd.Normal(a_bar, sigma), sample_shape=48)
    # Convert data to JAX arrays
    density_jax = jnp.array(d.density.values, dtype=jnp.float32)
    tank_jax = jnp.array(d.tank.values, dtype=jnp.float32)
    yield tfd.Independent(tfd.Binomial(total_count=density_jax,
                                        logits=jnp.squeeze(jnp.take(alpha, jnp.asarray(tank_jax, dtype=jnp.int32), axis=-1)),
                                        name='main'),
                          reinterpreted_batch_ndims=1)




dist = tfd.JointDistributionCoroutine(mymodel)

observed_data = d.surv.astype('float32').values
observed_data

def target_log_prob(*params):
  return dist.log_prob(params + (observed_data,))

init_key, sample_key = random.split(random.PRNGKey(0))
init_params = tuple(dist.sample(seed=jnp.array(init_key, dtype=jnp.uint32))[:-1])

@jit
def run_chain(key, state):
  kernel = tfp.mcmc.NoUTurnSampler(target_log_prob, 1e-3)
  return tfp.mcmc.sample_chain(500,
      current_state=state,
      kernel=kernel,
      trace_fn=lambda _, results: results.target_log_prob,
      num_burnin_steps=500,
      seed=key)
start = tm.time() 
states, log_probs = run_chain(sample_key, init_params)
end = tm.time()    
print(f"TF took: {end - start:.4f} seconds")


TF took: 1.9554 seconds


In [21]:
states

(Array([[1.3962194 ],
        [1.0456426 ],
        [1.52939   ],
        [1.1513127 ],
        [1.2277997 ],
        [1.5241737 ],
        [1.1897937 ],
        [1.1438453 ],
        [1.4381316 ],
        [1.5066295 ],
        [1.3738824 ],
        [1.3103633 ],
        [1.2593637 ],
        [1.153846  ],
        [1.6264533 ],
        [1.3773836 ],
        [1.6359632 ],
        [1.7549192 ],
        [1.6351682 ],
        [1.3829409 ],
        [1.4341145 ],
        [1.6117067 ],
        [1.0963628 ],
        [1.532092  ],
        [0.9939046 ],
        [1.7163693 ],
        [1.0947424 ],
        [1.5189432 ],
        [1.4087733 ],
        [1.4301178 ],
        [1.641571  ],
        [1.1981555 ],
        [1.4289936 ],
        [1.3707258 ],
        [1.5295569 ],
        [1.3286783 ],
        [1.4623374 ],
        [1.4131473 ],
        [1.4722674 ],
        [1.3858641 ],
        [1.427299  ],
        [1.2142785 ],
        [1.4200373 ],
        [1.2267808 ],
        [1.143426  ],
        [1